In [1]:
!git clone https://github.com/bearpaw/pytorch-classification.git

Cloning into 'pytorch-classification'...
remote: Enumerating objects: 287, done.
Receiving objects: 100% (287/287), 440.37 KiB | 15.73 MiB/s, done.
remote: Total 287 (delta 0), reused 0 (delta 0), pack-reused 287 (from 1)
Resolving deltas: 100% (167/167), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/Shareddrives/FML_Project/

/content/drive/Shareddrives/FML_Project


In [4]:
!unzip /content/drive/Shareddrives/FML_Project/Project.zip -d /content/

Archive:  /content/drive/Shareddrives/FML_Project/Project.zip
   creating: /content/Project/
   creating: /content/Project/resnet164Cifar100/
  inflating: /content/Project/resnet164Cifar100/checkpoint.pth.tar
  inflating: /content/Project/resnet164Cifar100/log.eps
  inflating: /content/Project/resnet164Cifar100/log.txt
  inflating: /content/Project/resnet164Cifar100/model_best.pth.tar
   creating: /content/Project/densenet190Cifar100/
  inflating: /content/Project/densenet190Cifar100/checkpoint.pth.tar
  inflating: /content/Project/densenet190Cifar100/log.txt
  inflating: /content/Project/densenet190Cifar100/model_best.pth.tar
   creating: /content/Project/WRNCifar100/
  inflating: /content/Project/WRNCifar100/checkpoint.pth.tar
  inflating: /content/Project/WRNCifar100/log.eps
  inflating: /content/Project/WRNCifar100/log.txt
  inflating: /content/Project/WRNCifar100/model_best.pth.tar
   creating: /content/Project/zip/
  inflating: /content/Project/zip/resnet-110.zip
  inflating: /co

In [5]:
%cd /content/Project

/content/Project


In [10]:
file={"username":"madhurthakkar","key":"abbbd2dbc89cb0c7dfaca74a00221bbb"}
import json
with open('kaggle.json', 'w') as fp:
    json.dump(file, fp)
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

print("\nKaggle API token configured successfully!")


Kaggle API token configured successfully!


In [11]:
%cd /content/

/content


In [12]:
!kaggle datasets download -d jutrera/stanford-car-dataset-by-classes-folder

!unzip -q stanford-car-dataset-by-classes-folder.zip

print("\nDataset downloaded and unzipped successfully.")

Dataset URL: https://www.kaggle.com/datasets/jutrera/stanford-car-dataset-by-classes-folder
License(s): other
100% 1.82G/1.83G [00:13<00:00, 239MB/s]
100% 1.83G/1.83G [00:13<00:00, 143MB/s]

Dataset downloaded and unzipped successfully.


In [13]:
%pwd

'/content'

In [15]:
print("Downloading birds400 dataset...")
!kaggle datasets download -d antoniozarauzmoreno/birds400

print("Unzipping dataset...")
!unzip -q birds400.zip
print("Dataset downloaded and unzipped successfully.")

!ls -l

Dataset URL: https://www.kaggle.com/datasets/antoniozarauzmoreno/birds400
License(s): unknown
100% 1.30G/1.30G [00:04<00:00, 297MB/s]
100% 1.30G/1.30G [00:04<00:00, 293MB/s]
Unzipping dataset...
Dataset downloaded and unzipped successfully.
total 3281460
-rw-r--r--  1 root root     225299 Oct  1  2019 anno_test.csv
-rw-r--r--  1 root root     228214 Oct  1  2019 anno_train.csv
drwxr-xr-x  5 root root       4096 Nov  9 12:10 birds400
-rw-r--r--  1 root root 1398909039 Apr 12  2022 birds400.zip
drwxr-xr-x  3 root root       4096 Nov  9 12:08 car_data
drwxr-xr-x  2 root root       4096 Nov  9 12:09 cars
drwxr-xr-x  2 root root       4096 Nov  9 12:07 cifar-10
drwxr-xr-x  2 root root       4096 Nov  9 12:06 cifar-100
drwx------  5 root root       4096 Nov  9 12:03 drive
-rw-r--r--  1 root root     374830 Nov  9 12:07 hb_cifar100.zip
-rw-r--r--  1 root root     185223 Nov  9 12:07 hb_cifar10.zip
-rw-r--r--  1 root root       5454 Oct  1  2019 names.csv
drwxr-xr-x 11 root root       4096 Oct

In [20]:
%cd pytorch-classification/
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from collections import OrderedDict
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

transform_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616)),
])

print("\nLoading and splitting CIFAR-10 data...")
try:
    full_test_dataset = datasets.CIFAR10(root="./data",
                                         train=False,
                                         download=True,
                                         transform=transform_cifar)
    val_size, test_size = 5000, 5000
    val_dataset, test_dataset = random_split(
        full_test_dataset, [val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)
    print(f"  -> Validation samples: {len(val_dataset)}")
    print(f"  -> Test samples:       {len(test_dataset)}")
except Exception as e:
    print(f" ERROR: Could not load CIFAR-10 data. {e}")
    raise

def get_predictions(model, loader, device, temp=1.0):
    model.eval()
    all_conf, all_corr = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x) / temp
            probs = F.softmax(logits, dim=1)
            conf, pred = torch.max(probs, 1)
            all_conf.extend(conf.cpu().numpy())
            all_corr.extend((pred == y).cpu().numpy())
    return np.array(all_conf), np.array(all_corr)

def calculate_ece(confidences, correct, n_bins=15):
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        bin_lower, bin_upper = bin_boundaries[i], bin_boundaries[i + 1]
        in_bin = (confidences > bin_lower) & (confidences <= bin_upper)
        prop_in_bin = np.mean(in_bin)
        if prop_in_bin > 0:
            acc_bin = np.mean(correct[in_bin])
            conf_bin = np.mean(confidences[in_bin])
            ece += np.abs(conf_bin - acc_bin) * prop_in_bin
    return ece * 100

def plot_reliability_diagram(confidences, correct, n_bins, model_name, suffix):
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers, bin_uppers = bin_boundaries[:-1], bin_boundaries[1:]
    bin_accs, bin_confs, bin_props = np.zeros(n_bins), np.zeros(n_bins), np.zeros(n_bins)
    for i, (low, up) in enumerate(zip(bin_lowers, bin_uppers)):
        in_bin = (confidences > low) & (confidences <= up)
        bin_props[i] = np.mean(in_bin)
        if bin_props[i] > 0:
            bin_accs[i] = np.mean(correct[in_bin])
            bin_confs[i] = np.mean(confidences[in_bin])

    plt.figure(figsize=(8, 7))
    plt.bar(bin_lowers + 1/(2*n_bins), bin_accs,
            width=1/n_bins*0.9, alpha=0.3, color="red", label="Accuracy")
    plt.plot([0, 1], [0, 1], "k--", label="Perfect Calibration")
    plt.xlabel("Confidence"); plt.ylabel("Accuracy")
    plt.title(f"Reliability Diagram: {model_name} ({suffix})")
    plt.legend(); plt.xlim(0, 1); plt.ylim(0, 1)
    save_folder = "/content/cifar-10"; os.makedirs(save_folder, exist_ok=True)
    fname = f"{save_folder}/{model_name.replace(' ', '_')}_C10_reliability_{suffix}.png"
    plt.savefig(fname); plt.close()
    print(f" Saved reliability diagram: {fname}")

def load_checkpoint(model, path, device):
    print(f"Loading checkpoint: {path}")
    ckpt = torch.load(path, map_location=device, weights_only=False)
    state_dict = ckpt.get("state_dict", ckpt)
    new_sd = OrderedDict((k.replace("module.", ""), v) for k, v in state_dict.items())
    model.load_state_dict(new_sd)
    return model

class VectorScaling(nn.Module):
    """Per-class temperature + bias calibration."""
    def __init__(self, num_classes=10, device="cuda"):
        super(VectorScaling, self).__init__()
        self.num_classes = num_classes
        self.device = device
        self.temperature = nn.Parameter(torch.ones(num_classes, device=device))
        self.bias = nn.Parameter(torch.zeros(num_classes, device=device))

    def forward(self, logits):
        return logits * self.temperature + self.bias

    def fit(self, model, val_loader, device):
        model.eval()
        self.to(device)

        logits_list, labels_list = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits_list.append(model(x))
                labels_list.append(y)

        logits = torch.cat(logits_list).to(device)
        labels = torch.cat(labels_list).to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.LBFGS(self.parameters(), lr=0.01, max_iter=150)

        def closure():
            optimizer.zero_grad()
            loss = criterion(self.forward(logits), labels)
            loss.backward()
            return loss

        optimizer.step(closure)
        with torch.no_grad():
            loss_final = criterion(self.forward(logits), labels).item()
        print(f" Vector Scaling optimized: mean(T)={self.temperature.mean().item():.4f}, "
              f"mean(b)={self.bias.mean().item():.4f}, NLL={loss_final:.4f}")

def run_ece_calibration(model, model_name, n_bins=15):
    print("\n" + "="*60)
    print(f" ECE Calibration for: {model_name} on CIFAR-10")
    print("="*60)
    model.to(device).eval()

    conf_b, corr_b = get_predictions(model, test_loader, device, temp=1.0)
    ece_b = calculate_ece(conf_b, corr_b, n_bins)
    acc_b, conf_avg_b = np.mean(corr_b)*100, np.mean(conf_b)*100
    print(f"\nBefore Calibration → Acc: {acc_b:.2f}%, Conf: {conf_avg_b:.2f}%, ECE: {ece_b:.3f}%")
    plot_reliability_diagram(conf_b, corr_b, n_bins, model_name, "before")

    vs = VectorScaling(num_classes=10, device=device)
    vs.fit(model, val_loader, device)

    all_conf, all_corr = [], []
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            logits = vs(model(x))
            probs = F.softmax(logits, dim=1)
            conf, pred = torch.max(probs, 1)
            all_conf.extend(conf.cpu().numpy())
            all_corr.extend((pred == y).cpu().numpy())

    conf_a, corr_a = np.array(all_conf), np.array(all_corr)
    ece_a = calculate_ece(conf_a, corr_a, n_bins)
    acc_a, conf_avg_a = np.mean(corr_a)*100, np.mean(conf_a)*100
    print(f"\nAfter Vector Scaling → Acc: {acc_a:.2f}%, Conf: {conf_avg_a:.2f}%, ECE: {ece_a:.3f}%\n")
    plot_reliability_diagram(conf_a, corr_a, n_bins, model_name, "after")

    return {
        "name": model_name,
        "acc": acc_a,
        "ece_before": ece_b,
        "ece_after": ece_a,
        "conf_before": conf_avg_b,
        "conf_after": conf_avg_a,
    }

all_results = []

try:
    from models.cifar import resnet
    print("\n--- Running calibration for local ResNet-164 ---")
    model_resnet164 = resnet(depth=164, num_classes=10, block_name="Bottleneck")
    path = "/content/Project/resnet110cifar10/model_best.pth.tar"
    model_resnet164 = load_checkpoint(model_resnet164, path, device)
    results = run_ece_calibration(model_resnet164, "ResNet-164")
    all_results.append(results)
except Exception as e:
    print(f" Skipping local ResNet-164: {e}")

try:
    print("\n--- Running calibration for ResNet-56 (torch.hub) ---")
    model_hub = torch.hub.load("chenyaofo/pytorch-cifar-models",
                               "cifar10_resnet56",
                               pretrained=True,
                               trust_repo=True)
    results = run_ece_calibration(model_hub, "ResNet-56 (Hub)")
    all_results.append(results)
except Exception as e:
    print(f" Skipping ResNet-56 (Hub): {e}")

if all_results:
    print("\n" + "="*130)
    print(" Final Calibration Comparison on CIFAR-10 Test Set (Vector Scaling)")
    print("="*130)
    print(f"{'Model':<22} | {'Accuracy':>10} | {'ECE (Before)':>13} | {'ECE (After)':>12} | "
          f"{'Conf (Before)':>15} | {'Conf (After)':>15}")
    print("-"*130)
    for r in all_results:
        print(f"{r['name']:<22} | {r['acc']:>9.2f}% | {r['ece_before']:>12.4f}% | "
              f"{r['ece_after']:>11.4f}% | {r['conf_before']:>14.2f}% | {r['conf_after']:>14.2f}%")
    print("="*130)
else:
    print("\nNo models were successfully calibrated.")


[Errno 2] No such file or directory: 'pytorch-classification/'
/content/pytorch-classification
Using device: cuda

Loading and splitting CIFAR-10 data...
  -> Validation samples: 5000
  -> Test samples:       5000

--- Running calibration for local ResNet-164 ---
Loading checkpoint: /content/Project/resnet110cifar10/model_best.pth.tar

ECE Calibration for: ResNet-164 on CIFAR-10

Before Calibration → Acc: 93.20%, Conf: 96.69%, ECE: 3.749%
Saved reliability diagram: /content/cifar-10/ResNet-164_C10_reliability_before.png
Vector Scaling optimized: mean(T)=0.6250, mean(b)=-0.0000, NLL=0.1947

After Vector Scaling → Acc: 93.42%, Conf: 94.05%, ECE: 1.184%

Saved reliability diagram: /content/cifar-10/ResNet-164_C10_reliability_after.png

--- Running calibration for ResNet-56 (torch.hub) ---

ECE Calibration for: ResNet-56 (Hub) on CIFAR-10


Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master



Before Calibration → Acc: 94.12%, Conf: 98.07%, ECE: 3.964%
Saved reliability diagram: /content/cifar-10/ResNet-56_(Hub)_C10_reliability_before.png
Vector Scaling optimized: mean(T)=0.6268, mean(b)=-0.0000, NLL=0.1907

After Vector Scaling → Acc: 94.02%, Conf: 95.56%, ECE: 1.625%

Saved reliability diagram: /content/cifar-10/ResNet-56_(Hub)_C10_reliability_after.png

Final Calibration Comparison on CIFAR-10 Test Set (Vector Scaling)
Model                  |   Accuracy |  ECE (Before) |  ECE (After) |   Conf (Before) |    Conf (After)
----------------------------------------------------------------------------------------------------------------------------------
ResNet-164             |     93.42% |       3.7487% |      1.1838% |          96.69% |          94.05%
ResNet-56 (Hub)        |     94.02% |       3.9644% |      1.6250% |          98.07% |          95.56%


In [29]:
%cd pytorch-classification/
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from collections import OrderedDict
import matplotlib.pyplot as plt
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

transform_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616)),
])

print("\nLoading and splitting CIFAR-100 data...")
try:
    full_test_dataset = datasets.CIFAR100(root="./data",
                                         train=False,
                                         download=True,
                                         transform=transform_cifar)
    val_size, test_size = 5000, 5000
    val_dataset, test_dataset = random_split(
        full_test_dataset, [val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)
    print(f"  -> Validation samples: {len(val_dataset)}")
    print(f"  -> Test samples:       {len(test_dataset)}")
except Exception as e:
    print(f" ERROR: Could not load CIFAR-100 data. {e}")
    raise

def get_predictions(model, loader, device, temp=1.0):
    model.eval()
    all_conf, all_corr = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x) / temp
            probs = F.softmax(logits, dim=1)
            conf, pred = torch.max(probs, 1)
            all_conf.extend(conf.cpu().numpy())
            all_corr.extend((pred == y).cpu().numpy())
    return np.array(all_conf), np.array(all_corr)

def calculate_ece(confidences, correct, n_bins=15):
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        bin_lower, bin_upper = bin_boundaries[i], bin_boundaries[i + 1]
        in_bin = (confidences > bin_lower) & (confidences <= bin_upper)
        prop_in_bin = np.mean(in_bin)
        if prop_in_bin > 0:
            acc_bin = np.mean(correct[in_bin])
            conf_bin = np.mean(confidences[in_bin])
            ece += np.abs(conf_bin - acc_bin) * prop_in_bin
    return ece * 100

def plot_reliability_diagram(confidences, correct, n_bins, model_name, suffix):
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers, bin_uppers = bin_boundaries[:-1], bin_boundaries[1:]
    bin_accs, bin_confs, bin_props = np.zeros(n_bins), np.zeros(n_bins), np.zeros(n_bins)
    for i, (low, up) in enumerate(zip(bin_lowers, bin_uppers)):
        in_bin = (confidences > low) & (confidences <= up)
        bin_props[i] = np.mean(in_bin)
        if bin_props[i] > 0:
            bin_accs[i] = np.mean(correct[in_bin])
            bin_confs[i] = np.mean(confidences[in_bin])

    plt.figure(figsize=(8, 7))
    plt.bar(bin_lowers + 1/(2*n_bins), bin_accs,
            width=1/n_bins*0.9, alpha=0.3, color="red", label="Accuracy")
    plt.plot([0, 1], [0, 1], "k--", label="Perfect Calibration")
    plt.xlabel("Confidence"); plt.ylabel("Accuracy")
    plt.title(f"Reliability Diagram: {model_name} ({suffix})")
    plt.legend(); plt.xlim(0, 1); plt.ylim(0, 1)
    save_folder = "/content/cifar-100"; os.makedirs(save_folder, exist_ok=True)
    fname = f"{save_folder}/{model_name.replace(' ', '_')}_C100_reliability_{suffix}.png"
    plt.savefig(fname); plt.close()
    print(f" Saved reliability diagram: {fname}")

def load_checkpoint(model, path, device):
    print(f"Loading checkpoint: {path}")
    ckpt = torch.load(path, map_location=device, weights_only=False)
    state_dict = ckpt.get("state_dict", ckpt)
    new_sd = OrderedDict((k.replace("module.", ""), v) for k, v in state_dict.items())
    model.load_state_dict(new_sd)
    return model

class VectorScaling(nn.Module):
    """Per-class temperature + bias calibration."""
    def __init__(self, num_classes=100, device="cuda"):
        super(VectorScaling, self).__init__()
        self.num_classes = num_classes
        self.device = device
        self.temperature = nn.Parameter(torch.ones(num_classes, device=device))
        self.bias = nn.Parameter(torch.zeros(num_classes, device=device))

    def forward(self, logits):
        return logits * self.temperature + self.bias

    def fit(self, model, val_loader, device):
        model.eval()
        self.to(device)

        logits_list, labels_list = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits_list.append(model(x))
                labels_list.append(y)

        logits = torch.cat(logits_list).to(device)
        labels = torch.cat(labels_list).to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.LBFGS(self.parameters(), lr=0.01, max_iter=150)

        def closure():
            optimizer.zero_grad()
            loss = criterion(self.forward(logits), labels)
            loss.backward()
            return loss

        optimizer.step(closure)
        with torch.no_grad():
            loss_final = criterion(self.forward(logits), labels).item()
        print(f" Vector Scaling optimized: mean(T)={self.temperature.mean().item():.4f}, "
              f"mean(b)={self.bias.mean().item():.4f}, NLL={loss_final:.4f}")

def run_vs_calibration(model, model_name, n_bins=15):
    print("\n" + "="*60)
    print(f" ECE Calibration for: {model_name} on CIFAR-100")
    print("="*60)
    model.to(device).eval()

    conf_b, corr_b = get_predictions(model, test_loader, device, temp=1.0)
    ece_b = calculate_ece(conf_b, corr_b, n_bins)
    acc_b, conf_avg_b = np.mean(corr_b)*100, np.mean(conf_b)*100
    print(f"\nBefore Calibration → Acc: {acc_b:.2f}%, Conf: {conf_avg_b:.2f}%, ECE: {ece_b:.3f}%")
    plot_reliability_diagram(conf_b, corr_b, n_bins, model_name, "before")

    vs = VectorScaling(num_classes=100, device=device)
    vs.fit(model, val_loader, device)

    all_conf, all_corr = [], []
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            logits = vs(model(x))
            probs = F.softmax(logits, dim=1)
            conf, pred = torch.max(probs, 1)
            all_conf.extend(conf.cpu().numpy())
            all_corr.extend((pred == y).cpu().numpy())

    conf_a, corr_a = np.array(all_conf), np.array(all_corr)
    ece_a = calculate_ece(conf_a, corr_a, n_bins)
    acc_a, conf_avg_a = np.mean(corr_a)*100, np.mean(conf_a)*100
    print(f"\nAfter Vector Scaling → Acc: {acc_a:.2f}%, Conf: {conf_avg_a:.2f}%, ECE: {ece_a:.3f}%\n")
    plot_reliability_diagram(conf_a, corr_a, n_bins, model_name, "after")

    return {
        "name": model_name,
        "acc": acc_a,
        "ece_before": ece_b,
        "ece_after": ece_a,
        "conf_before": conf_avg_b,
        "conf_after": conf_avg_a,
    }

all_results = []

try:
    from models.cifar import resnet
    from models.cifar.densenet import densenet, Bottleneck
    from models.cifar.wrn import WideResNet
    print("\n--- Running calibration for local ResNet-164 ---")
    model_resnet164 = resnet(depth=164, num_classes=100, block_name="Bottleneck")
    path = "/content/Project/resnet164Cifar100/model_best.pth.tar"
    model_resnet164 = load_checkpoint(model_resnet164, path, device)
    results = run_vs_calibration(model_resnet164, "ResNet-164")
    all_results.append(results)
except Exception as e:
    print(f" Skipping local ResNet-164: {e}")

try:
    print("\n--- Running calibration for ResNet-56 (torch.hub) ---")
    model_hub = torch.hub.load("chenyaofo/pytorch-cifar-models",
                               "cifar100_resnet56",
                               pretrained=True,
                               trust_repo=True)
    results = run_vs_calibration(model_hub, "ResNet-56 (Hub)")
    all_results.append(results)
except Exception as e:
    print(f" Skipping ResNet-56 (Hub): {e}")

try:
    print("\n--- Running calibration for WideResNet-28-10 ---")
    model_wrn = WideResNet(depth=28, num_classes=100, widen_factor=10, dropRate=0.3)
    path = '/content/Project/WRNCifar100/model_best.pth.tar'
    model_wrn = load_checkpoint(model_wrn, path, device)
    results = run_vs_calibration(model_wrn, "WideResNet-28-10")
    all_results.append(results)
except Exception as e:
    print(f" Skipping WideResNet-28-10: {e}")

try:
    print("\n--- Running calibration for DenseNet-190 ---")
    model_densenet = densenet(depth=190, num_classes=100, growthRate=40,
                                       compressionRate=2, block=Bottleneck)
    path = '/content/Project/densenet190Cifar100/model_best.pth.tar'
    model_densenet = load_checkpoint(model_densenet, path, device)
    results = run_vs_calibration(model_densenet, "DenseNet-190")
    all_results.append(results)
except Exception as e:
    print(f" Skipping DenseNet-190: {e}")


if all_results:
    print("\n" + "="*130)
    print(" Final Calibration Comparison on CIFAR-100 Test Set (Vector Scaling)")
    print("="*130)
    print(f"{'Model':<22} | {'Accuracy':>10} | {'ECE (Before)':>13} | {'ECE (After)':>12} | "
          f"{'Conf (Before)':>15} | {'Conf (After)':>15}")
    print("-"*130)
    for r in all_results:
        print(f"{r['name']:<22} | {r['acc']:>9.2f}% | {r['ece_before']:>12.4f}% | "
              f"{r['ece_after']:>11.4f}% | {r['conf_before']:>14.2f}% | {r['conf_after']:>14.2f}%")
    print("="*130)
else:
    print("\nNo models were successfully calibrated.")

[Errno 2] No such file or directory: 'pytorch-classification/'
/content/pytorch-classification
Using device: cuda

Loading and splitting CIFAR-100 data...
  -> Validation samples: 5000
  -> Test samples:       5000

--- Running calibration for local ResNet-164 ---
Loading checkpoint: /content/Project/resnet164Cifar100/model_best.pth.tar

ECE Calibration for: ResNet-164 on CIFAR-100

Before Calibration → Acc: 72.34%, Conf: 83.08%, ECE: 10.739%
Saved reliability diagram: /content/cifar-100/ResNet-164_C100_reliability_before.png
Vector Scaling optimized: mean(T)=0.6930, mean(b)=-0.0000, NLL=0.9077

After Vector Scaling → Acc: 72.98%, Conf: 74.96%, ECE: 1.988%

Saved reliability diagram: /content/cifar-100/ResNet-164_C100_reliability_after.png

--- Running calibration for ResNet-56 (torch.hub) ---

ECE Calibration for: ResNet-56 (Hub) on CIFAR-100


Using cache found in /root/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master



Before Calibration → Acc: 72.06%, Conf: 86.10%, ECE: 14.042%
Saved reliability diagram: /content/cifar-100/ResNet-56_(Hub)_C100_reliability_before.png
Vector Scaling optimized: mean(T)=0.6067, mean(b)=0.0000, NLL=1.0156

After Vector Scaling → Acc: 72.00%, Conf: 74.53%, ECE: 3.448%

Saved reliability diagram: /content/cifar-100/ResNet-56_(Hub)_C100_reliability_after.png

--- Running calibration for WideResNet-28-10 ---
Loading checkpoint: /content/Project/WRNCifar100/model_best.pth.tar

ECE Calibration for: WideResNet-28-10 on CIFAR-100

Before Calibration → Acc: 80.94%, Conf: 87.21%, ECE: 6.738%
Saved reliability diagram: /content/cifar-100/WideResNet-28-10_C100_reliability_before.png
Vector Scaling optimized: mean(T)=0.8670, mean(b)=-0.0000, NLL=0.7255

After Vector Scaling → Acc: 81.10%, Conf: 83.68%, ECE: 4.270%

Saved reliability diagram: /content/cifar-100/WideResNet-28-10_C100_reliability_after.png

--- Running calibration for DenseNet-190 ---
Loading checkpoint: /content/Proje

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import OrderedDict

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"==> Using device: {device}")

data_transforms = {
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
}

print("\nLoading and splitting data...")
CAR_TEST_PATH = '/content/car_data/car_data/test'

try:
    full_test_dataset = datasets.ImageFolder(CAR_TEST_PATH, data_transforms['test'])
    num_classes = len(full_test_dataset.classes)
    print(f"Found {num_classes} classes in the dataset.")

    val_size = 4000
    test_size = len(full_test_dataset) - val_size

    if test_size <= 0:
        raise ValueError("Validation size exceeds dataset size!")

    val_dataset, test_dataset = random_split(
        full_test_dataset,
        [val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    print(f"Validation samples: {len(val_dataset)}")
    print(f"Test samples:       {len(test_dataset)}")

except FileNotFoundError:
    print(f" ERROR: Data directory not found at: {CAR_TEST_PATH}")
    print("Please make sure you ran the data preparation cell first.")
except Exception as e:
    print(f" An error occurred during data loading: {e}")


def get_predictions(model, loader, device, temp=1.0):
    model.eval()
    all_conf, all_corr = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x) / temp
            probs = F.softmax(logits, dim=1)
            conf, pred = torch.max(probs, 1)
            all_conf.extend(conf.cpu().numpy())
            all_corr.extend((pred == y).cpu().numpy())
    return np.array(all_conf), np.array(all_corr)


def calculate_ece(confidences, correct, n_bins=15):
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        bin_lower, bin_upper = bin_boundaries[i], bin_boundaries[i + 1]
        in_bin = (confidences > bin_lower) & (confidences <= bin_upper)
        prop_in_bin = np.mean(in_bin)
        if prop_in_bin > 0:
            acc_bin = np.mean(correct[in_bin])
            conf_bin = np.mean(confidences[in_bin])
            ece += np.abs(conf_bin - acc_bin) * prop_in_bin
    return ece * 100


def plot_reliability_diagram(confidences, correct, n_bins, model_name, suffix):
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers, bin_uppers = bin_boundaries[:-1], bin_boundaries[1:]
    bin_accs, bin_confs, bin_props = np.zeros(n_bins), np.zeros(n_bins), np.zeros(n_bins)
    for i, (low, up) in enumerate(zip(bin_lowers, bin_uppers)):
        in_bin = (confidences > low) & (confidences <= up)
        bin_props[i] = np.mean(in_bin)
        if bin_props[i] > 0:
            bin_accs[i] = np.mean(correct[in_bin])
            bin_confs[i] = np.mean(confidences[in_bin])

    plt.figure(figsize=(8, 7))
    plt.bar(bin_lowers + 1/(2*n_bins), bin_accs,
            width=1/n_bins*0.9, alpha=0.3, color="red", label="Accuracy")
    plt.plot([0, 1], [0, 1], "k--", label="Perfect Calibration")
    plt.xlabel("Confidence"); plt.ylabel("Accuracy")
    plt.title(f"Reliability Diagram: {model_name} ({suffix})")
    plt.legend(); plt.xlim(0, 1); plt.ylim(0, 1)
    save_folder = "/content/cars"; os.makedirs(save_folder, exist_ok=True)
    fname = f"{save_folder}/{model_name.replace(' ', '_')}_reliability_{suffix}.png"
    plt.savefig(fname); plt.close()
    print(f" Saved reliability diagram: {fname}")


class VectorScaling(nn.Module):
    """Per-class temperature + bias calibration."""
    def __init__(self, num_classes, device="cuda"):
        super(VectorScaling, self).__init__()
        self.num_classes = num_classes
        self.device = device
        self.temperature = nn.Parameter(torch.ones(num_classes, device=device))
        self.bias = nn.Parameter(torch.zeros(num_classes, device=device))

    def forward(self, logits):
        return logits * self.temperature + self.bias

    def fit(self, model, val_loader, device):
        model.eval()
        self.to(device)

        logits_list, labels_list = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits_list.append(model(x))
                labels_list.append(y)

        logits = torch.cat(logits_list).to(device)
        labels = torch.cat(labels_list).to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.LBFGS(self.parameters(), lr=0.01, max_iter=150)

        def closure():
            optimizer.zero_grad()
            loss = criterion(self.forward(logits), labels)
            loss.backward()
            return loss

        optimizer.step(closure)
        with torch.no_grad():
            loss_final = criterion(self.forward(logits), labels).item()
        print(f" Vector Scaling optimized: mean(T)={self.temperature.mean().item():.4f}, "
              f"mean(b)={self.bias.mean().item():.4f}, NLL={loss_final:.4f}")


def run_vs_calibration(model, model_name, num_classes, n_bins=15):
    print("\n" + "=" * 70)
    print(f" Vector Scaling Calibration for: {model_name} (Stanford Cars)")
    print("=" * 70)

    model.to(device).eval()

    conf_b, corr_b = get_predictions(model, test_loader, device, temp=1.0)
    acc_b = np.mean(corr_b) * 100
    ece_b = calculate_ece(conf_b, corr_b, n_bins)
    avg_conf_b = np.mean(conf_b) * 100
    print(f"\nBefore Calibration → Acc: {acc_b:.2f}%, Conf: {avg_conf_b:.2f}%, ECE: {ece_b:.3f}%")
    plot_reliability_diagram(conf_b, corr_b, n_bins, model_name, "before_vs")

    vs = VectorScaling(num_classes=num_classes, device=device)
    vs.fit(model, val_loader, device)
    print(f" Vector Scaling fitted on validation data ({num_classes} classes)")

    all_conf_a, all_corr_a = [], []
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            logits = vs(model(x))
            probs = F.softmax(logits, dim=1)
            conf, pred = torch.max(probs, 1)
            all_conf_a.extend(conf.cpu().numpy())
            all_corr_a.extend((pred == y).cpu().numpy())

    conf_a, corr_a = np.array(all_conf_a), np.array(all_corr_a)
    ece_a = calculate_ece(conf_a, corr_a, n_bins)
    avg_conf_a = np.mean(conf_a) * 100

    acc_a = np.mean(corr_a) * 100

    print(f"\nAfter Vector Scaling → Acc: {acc_a:.2f}%, Conf: {avg_conf_a:.2f}%, ECE: {ece_a:.3f}%")
    plot_reliability_diagram(conf_a, corr_a, n_bins, model_name, "after_vs")

    return {
        "name": model_name,
        "acc_before_vs": acc_b,
        "acc_after_vs": acc_a,
        "ece_before": ece_b,
        "ece_after": ece_a,
        "conf_before": avg_conf_b,
        "conf_after": avg_conf_a,
    }


all_results = []
try:
    print("\nRe-creating and loading MobileNetV2 model...")
    base_model = models.mobilenet_v2(weights=None)

    if 'num_classes' not in locals():
        raise NameError("num_classes not defined")

    print(f"Building model for {num_classes} classes.")
    in_features = base_model.classifier[1].in_features
    base_model.classifier[1] = nn.Linear(in_features, num_classes)
    base_model.to(device)

    MODEL_PATH = '/content/Project/MobilenetV2_Cars/model_best.pth'
    base_model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    print(" Model loaded successfully.")

    results = run_vs_calibration(base_model, "MobileNetV2", num_classes=num_classes)
    all_results.append(results)

except FileNotFoundError:
    print(f" CRITICAL ERROR: Could not find model at '{MODEL_PATH}'")
except Exception as e:
    print(f" An error occurred: {e}")

if all_results:
    print("\n" + "=" * 140)
    print(" Final Vector Scaling Calibration Comparison on Stanford Cars Test Set")
    print("=" * 140)
    print(f"{'Model':<22} | {'Acc (Before VS)':>15} | {'Acc (After VS)':>14} | {'ECE (Before)':>13} | {'ECE (After)':>12} | {'Conf (Before)':>15} | {'Conf (After)':>14}")
    print("-" * 140)
    for r in all_results:
        print(f"{r['name']:<22} | {r['acc_before_vs']:>14.2f}% | {r['acc_after_vs']:>13.2f}% | {r['ece_before']:>12.4f}% | {r['ece_after']:>11.4f}% | {r['conf_before']:>14.2f}% | {r['conf_after']:>13.2f}%")
    print("=" * 140)
else:
    print("\nNo models were successfully calibrated to display a final table.")

==> Using device: cuda

Loading and splitting data...
Found 196 classes in the dataset.
Validation samples: 4000
Test samples:       4041

Re-creating and loading MobileNetV2 model...
Building model for 196 classes.
Model loaded successfully.

Vector Scaling Calibration for: MobileNetV2 (Stanford Cars)

Before Calibration → Acc: 44.35%, Conf: 53.77%, ECE: 9.421%
Saved reliability diagram: /content/cars/MobileNetV2_reliability_before_vs.png
Vector Scaling optimized: mean(T)=0.9080, mean(b)=0.0000, NLL=2.0478
Vector Scaling fitted on validation data (196 classes)

After Vector Scaling → Acc: 48.53%, Conf: 49.45%, ECE: 2.938%
Saved reliability diagram: /content/cars/MobileNetV2_reliability_after_vs.png

Final Vector Scaling Calibration Comparison on Stanford Cars Test Set
Model                  | Acc (Before VS) | Acc (After VS) |  ECE (Before) |  ECE (After) |   Conf (Before) |   Conf (After)
------------------------------------------------------------------------------------------------

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import OrderedDict
import torchvision

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"==> Using device: {device}")

data_transforms = {
    'test': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225]),
    ])
}

print("\nLoading and splitting data...")
BIRDS_TEST_PATH = '/content/birds400/test'

try:
    full_test_dataset = datasets.ImageFolder(BIRDS_TEST_PATH, data_transforms['test'])
    num_classes = len(full_test_dataset.classes)
    print(f"Found {num_classes} classes in the dataset.")

    val_size = 1000
    if len(full_test_dataset) <= val_size:
        print(f" ERROR: Total test set ({len(full_test_dataset)}) is too small for validation size ({val_size}).")
        exit()
    test_size = len(full_test_dataset) - val_size

    val_dataset, test_dataset = random_split(full_test_dataset, [val_size, test_size],
                                             generator=torch.Generator().manual_seed(42))

    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

    print(f"Data successfully split:")
    print(f"  -> Validation samples: {len(val_dataset)}")
    print(f"  -> Test samples:       {len(test_dataset)}")

except FileNotFoundError:
    print(f" ERROR: Data directory not found at: {BIRDS_TEST_PATH}")
    print("Please make sure you ran the data preparation cell first.")
except Exception as e:
    print(f" An error occurred during data loading: {e}")

def get_predictions(model, loader, device, temp=1.0):
    model.eval()
    all_conf, all_corr = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits = model(inputs)
            if isinstance(logits, tuple):
                logits = logits.logits
            logits = logits / temp
            probs = F.softmax(logits, dim=1)
            conf, pred = torch.max(probs, 1)
            all_conf.extend(conf.cpu().numpy())
            all_corr.extend((pred == labels).cpu().numpy())
    return np.array(all_conf), np.array(all_corr)

def calculate_ece(confidences, correct, n_bins=15):
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    ece = 0.0
    for i in range(n_bins):
        bin_lower, bin_upper = bin_boundaries[i], bin_boundaries[i + 1]
        in_bin = (confidences > bin_lower) & (confidences <= bin_upper)
        prop_in_bin = np.mean(in_bin)
        if prop_in_bin > 0:
            acc_bin = np.mean(correct[in_bin])
            conf_bin = np.mean(confidences[in_bin])
            ece += np.abs(conf_bin - acc_bin) * prop_in_bin
    return ece * 100

def plot_reliability_diagram(confidences, correct, n_bins, model_name, suffix):
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    bin_lowers, bin_uppers = bin_boundaries[:-1], bin_boundaries[1:]
    bin_accs, bin_confs, bin_props = np.zeros(n_bins), np.zeros(n_bins), np.zeros(n_bins)
    for i, (low, up) in enumerate(zip(bin_lowers, bin_uppers)):
        in_bin = (confidences > low) & (confidences <= up)
        bin_props[i] = np.mean(in_bin)
        if bin_props[i] > 0:
            bin_accs[i] = np.mean(correct[in_bin])
            bin_confs[i] = np.mean(confidences[in_bin])

    plt.figure(figsize=(8, 7))
    plt.bar(bin_lowers + 1/(2*n_bins), bin_accs,
            width=1/n_bins*0.9, alpha=0.3, color="red", label="Accuracy")
    plt.plot([0, 1], [0, 1], "k--", label="Perfect Calibration")
    plt.xlabel("Confidence"); plt.ylabel("Accuracy")
    plt.title(f"Reliability Diagram: {model_name} ({suffix})")
    plt.legend(); plt.xlim(0, 1); plt.ylim(0, 1)
    save_folder = "/content/birds"; os.makedirs(save_folder, exist_ok=True)
    fname = f"{save_folder}/{model_name.replace(' ', '_')}_reliability_{suffix}.png"
    plt.savefig(fname); plt.close()
    print(f" Saved reliability diagram: {fname}")

class VectorScaling(nn.Module):
    """Per-class temperature + bias calibration."""
    def __init__(self, num_classes, device="cuda"):
        super(VectorScaling, self).__init__()
        self.num_classes = num_classes
        self.device = device
        self.temperature = nn.Parameter(torch.ones(num_classes, device=device))
        self.bias = nn.Parameter(torch.zeros(num_classes, device=device))

    def forward(self, logits):
        return logits * self.temperature + self.bias

    def fit(self, model, val_loader, device):
        model.eval()
        self.to(device)

        logits_list, labels_list = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                if isinstance(logits, tuple):
                    logits = logits.logits
                logits_list.append(logits)
                labels_list.append(y)

        logits = torch.cat(logits_list).to(device)
        labels = torch.cat(labels_list).to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.LBFGS(self.parameters(), lr=0.01, max_iter=150)

        def closure():
            optimizer.zero_grad()
            loss = criterion(self.forward(logits), labels)
            loss.backward()
            return loss

        optimizer.step(closure)
        with torch.no_grad():
            loss_final = criterion(self.forward(logits), labels).item()
        print(f" Vector Scaling optimized: mean(T)={self.temperature.mean().item():.4f}, "
              f"mean(b)={self.bias.mean().item():.4f}, NLL={loss_final:.4f}")


def run_vs_calibration(model, model_name, num_classes, n_bins=15):
    print("\n" + "=" * 70)
    print(f" Vector Scaling Calibration for: {model_name} (Birds 400)")
    print("=" * 70)

    model.to(device).eval()

    conf_b, corr_b = get_predictions(model, test_loader, device, temp=1.0)
    acc_b = np.mean(corr_b) * 100
    ece_b = calculate_ece(conf_b, corr_b, n_bins)
    avg_conf_b = np.mean(conf_b) * 100
    print(f"\nBefore Calibration → Acc: {acc_b:.2f}%, Conf: {avg_conf_b:.2f}%, ECE: {ece_b:.3f}%")
    plot_reliability_diagram(conf_b, corr_b, n_bins, model_name, "before_vs")

    vs = VectorScaling(num_classes=num_classes, device=device)
    vs.fit(model, val_loader, device)
    print(f" Vector Scaling fitted on validation data ({num_classes} classes)")

    all_conf_a, all_corr_a = [], []
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            logits = vs(model(x))
            if isinstance(model(x), tuple):
                 logits = vs(model(x).logits)
            else:
                 logits = vs(model(x))
            probs = F.softmax(logits, dim=1)
            conf, pred = torch.max(probs, 1)
            all_conf_a.extend(conf.cpu().numpy())
            all_corr_a.extend((pred == y).cpu().numpy())

    conf_a, corr_a = np.array(all_conf_a), np.array(all_corr_a)
    ece_a = calculate_ece(conf_a, corr_a, n_bins)
    avg_conf_a = np.mean(conf_a) * 100

    acc_a = np.mean(corr_a) * 100

    print(f"\nAfter Vector Scaling → Acc: {acc_a:.2f}%, Conf: {avg_conf_a:.2f}%, ECE: {ece_a:.3f}%")
    plot_reliability_diagram(conf_a, corr_a, n_bins, model_name, "after_vs")

    return {
        "name": model_name,
        "acc_before_vs": acc_b,
        "acc_after_vs": acc_a,
        "ece_before": ece_b,
        "ece_after": ece_a,
        "conf_before": avg_conf_b,
        "conf_after": avg_conf_a,
    }

all_results = []
try:
    if 'num_classes' not in locals():
        raise NameError("num_classes not defined")

    model_name = "InceptionV3_Fold9"
    MODEL_PATH = "/content/Project/InceptionNetV3_Birds/inceptionv3_birds9.pth"

    print("\n" + "="*80)
    print(f"STARTING VECTOR SCALING FOR: {model_name} from {MODEL_PATH}")
    print("="*80)

    torch.serialization.add_safe_globals([torchvision.models.inception.Inception3])
    loaded_object = torch.load(MODEL_PATH, map_location=device, weights_only=False)

    if isinstance(loaded_object, nn.Module):
        base_model = loaded_object
        if hasattr(base_model, 'fc') and isinstance(base_model.fc, nn.Linear):
             in_features = base_model.fc.in_features
             if base_model.fc.out_features != num_classes:
                 print(f"Adjusting final layer from {base_model.fc.out_features} to {num_classes} classes.")
                 base_model.fc = nn.Linear(in_features, num_classes)
        elif hasattr(base_model, 'AuxLogits') and hasattr(base_model.AuxLogits, 'fc') and isinstance(base_model.AuxLogits.fc, nn.Linear):
             in_features_aux = base_model.AuxLogits.fc.in_features
             if base_model.AuxLogits.fc.out_features != num_classes:
                 print(f"Adjusting aux layer from {base_model.AuxLogits.fc.out_features} to {num_classes} classes.")
                 base_model.AuxLogits.fc = nn.Linear(in_features_aux, num_classes)
        else:
             print("Warning: Could not automatically detect and adjust final layer for class count.")


        base_model.to(device)
    elif isinstance(loaded_object, dict):
        base_model = models.inception_v3(weights=None, aux_logits=True, init_weights=False)
        in_features = base_model.fc.in_features
        base_model.fc = nn.Linear(in_features, num_classes)
        if base_model.AuxLogits is not None:
             in_features_aux = base_model.AuxLogits.fc.in_features
             base_model.AuxLogits.fc = nn.Linear(in_features_aux, num_classes)

        base_model.to(device)
        state_dict = loaded_object.get('state_dict', loaded_object)
        new_sd = OrderedDict((k.replace('module.', ''), v) for k, v in state_dict.items())

        model_dict = base_model.state_dict()
        pretrained_dict = {k: v for k, v in new_sd.items() if k in model_dict and model_dict[k].shape == v.shape}
        model_dict.update(pretrained_dict)
        base_model.load_state_dict(model_dict)

    else:
        raise TypeError(f"Unexpected object type: {type(loaded_object)}")

    print("Model loaded successfully.")

    base_model.eval()

    results = run_vs_calibration(base_model, model_name, num_classes=num_classes)
    all_results.append(results)

except FileNotFoundError:
    print(f" CRITICAL ERROR: Could not find weights at '{MODEL_PATH}'")
except NameError as e:
     print(f" CRITICAL ERROR: {e}. Make sure num_classes is defined by running the data loading section.")
except Exception as e:
    print(f" An error occurred: {e}")

print("\n" + "="*140)
print(" Final Vector Scaling Calibration Results on Birds Dataset Test Set")
print("="*140)
print(f"{'Model':<22} | {'Acc (Before VS)':>15} | {'Acc (After VS)':>14} | {'ECE (Before)':>13} | {'ECE (After)':>12} | {'Conf (Before)':>15} | {'Conf (After)':>14}")
print("-" * 140)
for r in all_results:
    print(f"{r['name']:<22} | {r['acc_before_vs']:>14.2f}% | {r['acc_after_vs']:>13.2f}% | {r['ece_before']:>12.4f}% | {r['ece_after']:>11.4f}% | {r['conf_before']:>14.2f}% | {r['conf_after']:>13.2f}%")
print("=" * 140)

==> Using device: cuda

Loading and splitting data...
Found 400 classes in the dataset.
Data successfully split:
  -> Validation samples: 1000
  -> Test samples:       1000

STARTING VECTOR SCALING FOR: InceptionV3_Fold9 from /content/Project/InceptionNetV3_Birds/inceptionv3_birds9.pth
Model loaded successfully.

Vector Scaling Calibration for: InceptionV3_Fold9 (Birds 400)

Before Calibration → Acc: 98.90%, Conf: 98.65%, ECE: 0.503%
Saved reliability diagram: /content/birds/InceptionV3_Fold9_reliability_before_vs.png
Vector Scaling optimized: mean(T)=1.0001, mean(b)=-0.0000, NLL=0.0183
Vector Scaling fitted on validation data (400 classes)

After Vector Scaling → Acc: 98.90%, Conf: 98.65%, ECE: 0.502%
Saved reliability diagram: /content/birds/InceptionV3_Fold9_reliability_after_vs.png

Final Vector Scaling Calibration Results on Birds Dataset Test Set
Model                  | Acc (Before VS) | Acc (After VS) |  ECE (Before) |  ECE (After) |   Conf (Before) |   Conf (After)
-----------

In [32]:
print("Creating a single zip file for all Vector Scaling results...")

zip_base_dir = "/content/vector_scaling_results"
os.makedirs(zip_base_dir, exist_ok=True)

cars_zip_dir = os.path.join(zip_base_dir, "cars")
birds_zip_dir = os.path.join(zip_base_dir, "birds")
cifar10_zip_dir = os.path.join(zip_base_dir, "cifar-10")
cifar100_zip_dir = os.path.join(zip_base_dir, "cifar-100")

os.makedirs(cars_zip_dir, exist_ok=True)
os.makedirs(birds_zip_dir, exist_ok=True)
os.makedirs(cifar10_zip_dir, exist_ok=True)
os.makedirs(cifar100_zip_dir, exist_ok=True)


cars_src_dir = "/content/cars"
if os.path.exists(cars_src_dir):
    !cp {cars_src_dir}/*_before_vs.png {cars_zip_dir}/
    !cp {cars_src_dir}/*_after_vs.png {cars_zip_dir}/
    print(" Added Stanford Cars VS results to zip structure.")

birds_src_dir = "/content/birds"
if os.path.exists(birds_src_dir):
    !cp {birds_src_dir}/*_before_vs.png {birds_zip_dir}/
    !cp {birds_src_dir}/*_after_vs.png {birds_zip_dir}/
    print(" Added Birds 400 VS results to zip structure.")

cifar10_src_dir = "/content/cifar-10"
if os.path.exists(cifar10_src_dir):
    !cp {cifar10_src_dir}/*_C10_reliability_before.png {cifar10_zip_dir}/
    !cp {cifar10_src_dir}/*_C10_reliability_after.png {cifar10_zip_dir}/
    print(" Added CIFAR-10 VS results to zip structure.")

cifar100_src_dir = "/content/cifar-100"
if os.path.exists(cifar100_src_dir):
    !cp {cifar100_src_dir}/*_C100_reliability_before.png {cifar100_zip_dir}/
    !cp {cifar100_src_dir}/*_C100_reliability_after.png {cifar100_zip_dir}/
    print(" Added CIFAR-100 VS results to zip structure.")


final_zip_path = "/content/vector_scaling_all_results.zip"
!zip -r {final_zip_path} {zip_base_dir}

print(f"\nCreated combined zip file: {final_zip_path}")

print("\nCopying the combined zip file to Google Drive...")
import os
if not os.path.exists('/content/drive'):
    print("Google Drive not mounted. Please mount it first.")
else:
    drive_destination_path = '/content/drive/Shareddrives/FML_Project/'
    os.makedirs(drive_destination_path, exist_ok=True)

    if os.path.exists(final_zip_path):
        !cp {final_zip_path} "{drive_destination_path}"
        print(f" Copied {os.path.basename(final_zip_path)} to Drive.")
    else:
        print(f" Error: Combined zip file not found at {final_zip_path}")

print("\nZipping and copying complete.")

Creating a single zip file for all Vector Scaling results...
Added Stanford Cars VS results to zip structure.
Added Birds 400 VS results to zip structure.
Added CIFAR-10 VS results to zip structure.
Added CIFAR-100 VS results to zip structure.
  adding: content/vector_scaling_results/ (stored 0%)
  adding: content/vector_scaling_results/cifar-100/ (stored 0%)
  adding: content/vector_scaling_results/cifar-100/ResNet-164_C100_reliability_after.png (deflated 15%)
  adding: content/vector_scaling_results/cifar-100/WideResNet-28-10_C100_reliability_after.png (deflated 13%)
  adding: content/vector_scaling_results/cifar-100/DenseNet-190_C100_reliability_before.png (deflated 15%)
  adding: content/vector_scaling_results/cifar-100/ResNet-56_(Hub)_C100_reliability_before.png (deflated 17%)
  adding: content/vector_scaling_results/cifar-100/DenseNet-190_C100_reliability_after.png (deflated 13%)
  adding: content/vector_scaling_results/cifar-100/WideResNet-28-10_C100_reliability_before.png (defl